In [8]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import sys
import glob
import pandas as pd
import os
import seaborn as sns

from tqdm import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
from pathlib import Path
import scipy.stats
import time

from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.examples.ubc_plotting_utils import *
from open_spiel.python.examples.ubc_sample_game_tree import sample_game_tree, flatten_trees, flatten_tree
from open_spiel.python.examples.ubc_clusters import projectPCA, fitGMM
from open_spiel.python.examples.ubc_utils import *
import open_spiel.python.examples.ubc_dispatch as dispatch

from auctions.webutils import *

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


from open_spiel.python.examples.ubc_cma import *

output_notebook()
from open_spiel.python.games.clock_auction_base import InformationPolicy
from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.examples.ubc_decorators import TakeSingleActionDecorator, TremblingAgentDecorator, ModalAgentDecorator

Loading BokehJS ...

In [3]:
# load runs from experiments
experiments = []
# experiments += ['jun14ent_3']
# experiments += ['jun13ppo']
# experiments += ['jun13outcome']
experiments += ['jun16outcome']
runs = []
for experiment in experiments:
    runs += Experiment.objects.get(name=experiment).equilibriumsolverrun_set.all()
print(f"Found {len(runs)} runs")

Found 75 runs


In [ ]:
def make_combo_specific(evaluation):
    r = dict()
    for k,v in evaluation.samples.items():
        if isinstance(v, dict):
            for k2, v2 in v.items():
                r[f'{k}_{k2}'] = v2
        else:
            r[k] = v
    del r['rewards']

    c_df = pd.DataFrame(r)
    type_columns = [c for c in c_df if 'types_' in c]
    c_df['combo'] = list(map(tuple, c_df[type_columns].values))
    mean_lengths = c_df.groupby('combo')['auction_lengths'].mean()
    ### A given type combo where tie-breaking (non-final) seems to matter
    for v in mean_lengths.values:
        if not v.is_integer() and v < 4:
            print("FOUND YOU", v)
            return True
    return False


In [6]:
for run in runs:
    try:
        for e in run.equilibriumsolverruncheckpoint_set.all():
            print(e.t, len(e.policy) / 1e9, len(str(e.get_old_eval().samples)) / 1e9)
        print()
    except:
        continue

3000000 0.005912499 0.001831277
6000000 0.007820413 0.001828757

3000000 0.127671561 0.001614566

3000000 0.020310832 0.002021805
3000000 0.13152797 0.001695781

3000000 0.012322041 0.002135664
6000000 0.016652617 0.002237279
3000000 0.038234417 0.00199949
3000000 0.002549552 0.001643515
6000000 0.003236882 0.001645749


3000000 0.004432026 0.002004528
6000000 0.00591497 0.001959486
3000000 0.004715055 0.001710424
6000000 0.006013676 0.001722405

3000000 0.004534938 0.001989627
6000000 0.005997323 0.001988385

3000000 0.001746455 0.00165457
6000000 0.002180123 0.001648194

3000000 0.131489253 0.001609162

3000000 0.035647432 0.002255889
3000000 0.019092255 0.001974475
6000000 0.023565506 0.001966577

3000000 0.006970396 0.001838616
6000000 0.009660307 0.001830287

3000000 0.002743604 0.001696669
3000000 0.008717741 0.002160857
6000000 0.012465132 0.002230682


3000000 0.004715068 0.001708916
6000000 0.006040473 0.001718471

3000000 0.002760429 0.002110017
6000000 0.003138169 0.00211190

In [35]:
len(e.get_model()._infostates)

15679

In [37]:
dir(e)

['DoesNotExist',
 'Meta',
 'MultipleObjectsReturned',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_column_name_clashes',
 '_check_constraints',
 '_check_default_pk',
 '_check_field_name_clashes',
 '_check_fields',
 '_check_id_field',
 '_check_index_together',
 '_check_indexes',
 '_check_local_fields',
 '_check_long_column_names',
 '_check_m2m_through_same_relationship',
 '_check_managers',
 '_check_model',
 '_check_model_name_db_lookup_clashes',
 '_check_ordering',
 '_check_property_name_related_field_accessor_clashes',
 '_check_single_primary_key',
 '_check_swappable',
 '_check_unique_together',
 '_do_insert',
 '_do_update'

# Analyze runs

In [16]:
# empirical = True
USE_MODAL = True

records = []
for run in tqdm(runs):
    try:
    
        try:
            game, final_checkpoint, policy = get_results(run, skip_single_chance_nodes=False)
        except Exception as e:
            print(f"Skipping run {run.name} because of error {e}")
            continue

        if USE_MODAL:
            evaluation = final_checkpoint.get_modal_eval()
            # if make_combo_specific(evaluation):
            #     print(run.name)
        else:
            evaluation = final_checkpoint.get_old_eval()
            
            
        # total_entropy = analyze_samples(final_checkpoint.get_old_eval().samples, game)['total_entropy'] # Use old eval since modal is stupid for this and it's a convergence test

        # if empirical: 
        #     history_dists = empirical_history_distribution(final_checkpoint.get_old_eval()) # Always use the old eval here
        #     history_entropies = [scipy.stats.entropy(list(history_dists.loc[type_combo].values)) for type_combo in np.unique(history_dists.index.get_level_values(0))]
        # else:
        #     history_dists = compute_per_type_combo(history_distribution, policy, game, min_prob=0.01, history_type='processed')
        #     history_entropies = [scipy.stats.entropy(list(history_dists[type_combo].values())) for type_combo in history_dists]

        # try:
        #     # TODO: Broken until you remove "Tremble" agents
        #     _, _, approx_nash_conv = find_best_checkpoint(run)
        # except:
        #     # TODO: Fix bug where this actually happens because your BRs failed to do better and you have an Empirical Nash Conv of 0, very different from NaN
        #     # ev_df = parse_run(run, None)
        #     # display(ev_df)
        #     approx_nash_conv = np.nan

        record = {
            # clock auction params
            'game_name': run.game.name, 
            'potential': run.config.get('potential_function', 'None'),
            'seed': run.config.get('seed'), 
            'run_name': run.name,
            'experiment': run.experiment.name,
            'config': run.get_config_name(),
            't': final_checkpoint.t,

            # CMA knobs
            'information_policy': InformationPolicy(game.auction_params.information_policy).name,
            'clock_speed': game.auction_params.increment,
            'base_game_name': '_'.join(run.game.name.split('/')[1].split('_')[:2]), # Stupid naming convention that will surely bite us later

            # solver information
            'alg': get_algorithm_from_run(run),
            'walltime': run.walltime(),

            # 'approx_nash_conv': approx_nash_conv,

            # metrics from eval
            # **analyze_samples(evaluation.samples, game),
            # 'total_entropy': total_entropy,

            # stats about history distribution
            # 'avg_distinct_histories': np.mean([len(d) for d in history_dists.values()]),
            # 'avg_history_entropy': np.mean(history_entropies),
        } 
        
        record['nash_conv'] = final_checkpoint.get_modal_eval().nash_conv

        # Slowwwwww you may want to comment this out when not using it
#         start = time.time()
#         c = final_checkpoint.equilibrium_solver_run.config
#         cfr = c.get('solver_type') == 'cfr'
#         env_and_policy = ppo_db_checkpoint_loader(final_checkpoint, cfr=cfr)
#         for player in range(game.num_players()):
#             env_and_policy.agents[player] = ModalAgentDecorator(env_and_policy.agents[player])
#         modal_policy = env_and_policy.make_policy()
        
#         worked, nc = time_bounded_run(30, nash_conv, game, modal_policy)
#         if not worked:
#             print("Aborted run because time")
#         record['nash_conv'] = nc if worked else np.nan
#         record['nash_conv_time'] = time.time() - start
#         print(time.time() - start)



        records.append(record)
    except Exception as e:
        print(f"Something wrong with {run}. Skipping {e}")
print(len(records))

  1%|▌                                           | 1/75 [00:00<00:11,  6.42it/s]

Something wrong with jun16_jun16_3_medium_speed-cfr_outcomemccfr_outcome-101 (jun16outcome). Skipping Evaluation matching query does not exist.
Skipping run jun16_jun16_4_no_activity-cfr_outcomemccfr_outcome-100 because of error None final checkpoint?


  5%|██▎                                         | 4/75 [00:05<01:44,  1.47s/it]

Something wrong with jun16_jun16_0_medium_speed-cfr_outcomemccfr_outcome-100 (jun16outcome). Skipping Evaluation matching query does not exist.


  8%|███▌                                        | 6/75 [00:11<02:15,  1.96s/it]

...

Something wrong with jun16_jun16_1_medium_speed-cfr_outcomemccfr_outcome-102 (jun16outcome). Skipping Evaluation matching query does not exist.
Something wrong with jun16_jun16_1_undersell_allowed-cfr_outcomemccfr_outcome-101 (jun16outcome). Skipping Evaluation matching query does not exist.
Skipping run jun16_jun16_4_high_speed-cfr_outcomemccfr_outcome-100 because of error None final checkpoint?
Something wrong with jun16_jun16_1_base-cfr_outcomemccfr_outcome-101 (jun16outcome). Skipping Evaluation matching query does not exist.
Something wrong with jun16_jun16_3_high_speed-cfr_outcomemccfr_outcome-100 (jun16outcome). Skipping Evaluation matching query does not exist.


 96%|█████████████████████████████████████████▎ | 72/75 [00:47<00:00,  9.68it/s]

Something wrong with jun16_jun16_1_medium_speed-cfr_outcomemccfr_outcome-100 (jun16outcome). Skipping Evaluation matching query does not exist.
Something wrong with jun16_jun16_3_medium_speed-cfr_outcomemccfr_outcome-100 (jun16outcome). Skipping Evaluation matching query does not exist.
Something wrong with jun16_jun16_3_high_speed-cfr_outcomemccfr_outcome-101 (jun16outcome). Skipping Evaluation matching query does not exist.


100%|███████████████████████████████████████████| 75/75 [00:47<00:00,  1.58it/s]

Something wrong with jun16_jun16_4_medium_speed-cfr_outcomemccfr_outcome-100 (jun16outcome). Skipping Evaluation matching query does not exist.
Something wrong with jun16_jun16_0_high_speed-cfr_outcomemccfr_outcome-100 (jun16outcome). Skipping Evaluation matching query does not exist.
7


In [17]:
df = pd.DataFrame.from_records(records)
# df['imperfect'] = df['game_name'].str.contains('imperfect')


In [18]:
df

,game_name,potential,seed,run_name,experiment,config,t,information_policy,clock_speed,base_game_name,alg,walltime,nash_conv
0,jun16/jun16_1_no_activity.json,None,101,jun16_jun16_1_no_activity-cfr_outcomemccfr_out...,jun16outcome,cfr_outcomemccfr_outcome,3000000,SHOW_DEMAND,0.3,jun16_1,cfr_outcome,10934.102578,19.3025
1,jun16/jun16_1_no_activity.json,None,102,jun16_jun16_1_no_activity-cfr_outcomemccfr_out...,jun16outcome,cfr_outcomemccfr_outcome,3000000,SHOW_DEMAND,0.3,jun16_1,cfr_outcome,12907.232714,24.8840
2,jun16/jun16_1_no_activity.json,None,100,jun16_jun16_1_no_activity-cfr_outcomemccfr_out...,jun16outcome,cfr_outcomemccfr_outcome,3000000,SHOW_DEMAND,0.3,jun16_1,cfr_outcome,10233.574260,22.0800
3,jun16/jun16_1_undersell_allowed.json,None,100,jun16_jun16_1_undersell_allowed-cfr_outcomemcc...,jun16outcome,cfr_outcomemccfr_outcome,6000000,SHOW_DEMAND,0.3,jun16_1,cfr_outcome,14917.998551,NaN
4,jun16/jun16_4_base.json,None,102,jun16_jun16_4_base-cfr_outcomemccfr_outcome-102,jun16outcome,cfr_outcomemccfr_outcome,6000000,SHOW_DEMAND,0.3,jun16_4,cfr_outcome,18339.957468,NaN
5,jun16/jun16_4_undersell_allowed.json,None,100,jun16_jun16_4_undersell_allowed-cfr_outcomemcc...,jun16outcome,cfr_outcomemccfr_outcome,6000000,SHOW_DEMAND,0.3,jun16_4,cfr_outcome,16401.935545,NaN
6,jun16/jun16_0_high_speed.json,None,101,jun16_jun16_0_high_speed-cfr_outcomemccfr_outc...,jun16outcome,cfr_outcomemccfr_outcome,6000000,SHOW_DEMAND,0.6,jun16_0,cfr_outcome,17614.483184,NaN


In [ ]:
df.groupby('config')['walltime'].describe() / 3600

In [ ]:
df.query('~imperfect')['config'].value_counts(dropna=False)

In [ ]:
df.query('~imperfect').groupby('config')['nash_conv'].describe()

In [ ]:
df.query('~nash_conv.isnull()').sample(10)

In [ ]:
df = _

In [ ]:
df['run_name'].values

In [ ]:
df.groupby('experiment')['avg_history_entropy'].describe()

In [ ]:
groupers = ['base_game_name', 'clock_speed', 'information_policy']
df.sort_values(groupers).set_index([df.index]+groupers, drop=True).to_csv('jun5_results.csv')

In [ ]:
df.sort_values(groupers).set_index([df.index]+groupers, drop=True)
df.to_csv('greg3.csv')

In [ ]:
palette = {
    "('SHOW_DEMAND', 0.3)": 'blue',
    "('SHOW_DEMAND', 0.45)": 'orange',
    "('SHOW_DEMAND', 0.6)": 'green'
}

In [ ]:
df.columns

In [ ]:
ATTRIBUTE = 'variant'
NORMALIZED = False
plot_type = 'scatter'
box_df = df.copy()
markers = ['o', 'X', '^']

box_df['variant'] = list(map(str,(zip(box_df.information_policy, box_df.clock_speed))))

# box_df = box_df.query('clock_speed == 0.3')
box_df = box_df.query('information_policy == "SHOW_DEMAND"')
box_df = box_df.sort_values(['clock_speed', 'game_name', 'config'])

STATS = ['total_revenue', 'total_welfare', 'auction_lengths', 'total_entropy']
# Normalize by the mean of the "base" version, so the scales line up better in the same graph
if NORMALIZED: 
    for stat in STATS:
        base = str(('SHOW_DEMAND', 0.3))
        base_stat = box_df.pivot_table(index=['base_game_name'], values=stat, aggfunc='mean', columns=ATTRIBUTE)
        box_df[f'normalized_{stat}'] = box_df.apply(lambda x: x[stat] / base_stat.loc[x['base_game_name'], base], axis=1)

for stat in STATS:
    if NORMALIZED:
        stat = f'normalized_{stat}'
    plt.figure(figsize=(20,10))
    if plot_type == 'box':
        ax = sns.boxplot(data=box_df, x='base_game_name', y=stat, hue=ATTRIBUTE)
        # ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
    else:
        strip_kwargs = dict(x='base_game_name', y=stat, hue=ATTRIBUTE, s=10, alpha=0.5, dodge=True, jitter=False, palette=palette)
        for i, config in enumerate(box_df['config'].unique()):
            # print(config, markers[i])
            ax = sns.stripplot(data=box_df.query(f'config == "{config}"'), marker=markers[i], **strip_kwargs)
            if i == 0:
                old_handles, old_labels = ax.get_legend_handles_labels()
            # ax = sns.stripplot(data=box_df.query('alg.str.contains("PPO") and imperfect'), marker='X', **strip_kwargs)
            # ax = sns.stripplot(data=box_df.query('alg.str.contains("cfr")'), marker='^', **strip_kwargs)
        ax.legend(old_handles, old_labels)

    plt.title(f"{stat}")

In [ ]:


# ### Code to rerun evals for a failed experiment
# from open_spiel.python.examples.ubc_dispatch import dispatch_eval_database
# exp = 'jun5outcome'

# for e in Evaluation.objects.filter(checkpoint__equilibrium_solver_run__experiment__name=exp):
#     experiment_name = e.checkpoint.equilibrium_solver_run.experiment.name
#     run_name = e.checkpoint.equilibrium_solver_run.name
#     t = e.checkpoint.t
#     dispatch_eval_database(experiment_name, run_name, t, None, None)
# print(Evaluation.objects.filter(checkpoint__equilibrium_solver_run__experiment__name=exp).delete())

In [ ]:
df.query('alg == "PPO"').groupby(['experiment', 'imperfect']).mean()

In [ ]:
records = []

for run in tqdm(runs):
    try:
        ev_df = parse_run(run, expected_additional_br=0)
        if ev_df is None:
            continue
        ev_df = ev_df.query('name == "modal"')
        if not ev_df.empty:
            val = ((ev_df['Regret'] / ev_df['Baseline']).mean() * 100)
            record = dict(val=val, alg=get_algorithm_from_run(run), game=run.game.name)
            records.append(record)
    except KeyError as e:
        continue
val_df = pd.DataFrame.from_records(records)

In [ ]:
# val_df.groupby(['game', 'alg']).describe()
val_df.groupby(['game', 'alg']).describe()

In [ ]:
# import open_spiel.python.examples.ubc_dispatch as dispatch
# for run in runs:
#     try:
#         game, final_checkpoint, policy = get_results(run, skip_single_chance_nodes=False)
#     except:
#         print("Skipping", run)
#         continue
#     final_checkpoint.bestresponse_set.all().delete()
#     for player in range(game.num_players()):
#         dispatch.dispatch_eval_database(run.experiment.name, run.name, final_checkpoint.t, player, 'modal') 
#         dispatch.dispatch_eval_database(run.experiment.name, run.name, final_checkpoint.t, player, 'tremble')  

In [ ]:
x = {'blue': 3, 'red': 1111, "string": 'tortoise'}
y = str(x)
y

In [ ]:
eval(y)

In [ ]:
for run in runs:
    try:
        game, final_checkpoint, policy = get_results(run, skip_single_chance_nodes=False)
    except Exception as e:
        print(f"Skipping run {run.name} because of error {e}")
        continue

    ev = final_checkpoint.get_modal_eval()
    if ev.nash_conv:
        continue
    else:
        ev.delete()
        br_mapping = {p: 'modal' for p in range(2)}
        dispatch.dispatch_eval_database(final_checkpoint.t, run.experiment.name, run.name, str(br_mapping))

In [8]:
game = pyspiel.load_game('python_clock_auction')

In [11]:
a = game.new_initial_state()

In [12]:
b = a.child(0).child(0)

In [13]:
a.auction_params

AuctionParams(opening_prices=[100, 100], licenses=array([1, 1]), license_names=['A', 'B'], activity=[100, 100], num_products=2, increment=0.1, reveal_type_round=-1, fold_randomness=True, skip_single_chance_nodes=True, max_round=6, player_types=defaultdict(<class 'list'>, {0: [{'prob': 1, 'bidder': <open_spiel.python.games.clock_auction_bidders.LinearBidder object at 0x7f9712e89e80>}], 1: [{'prob': 1, 'bidder': <open_spiel.python.games.clock_auction_bidders.LinearBidder object at 0x7f9712e89a30>}]}), all_bids=array([[0, 0],
       [0, 1],
       [1, 0],
       [1, 1]]), bid_to_index={(0, 0): 0, (0, 1): 1, (1, 0): 2, (1, 1): 3}, all_bids_activity=array([  0, 100, 100, 200]), activity_policy=<ActivityPolicy.ON: 0>, undersell_policy=<UndersellPolicy.UNDERSELL: 0>, information_policy=<InformationPolicy.SHOW_DEMAND: 0>, agent_memory=1)

In [25]:
id(a.auction_params.player_types[0][0]['bidder'])

140286834024064

In [26]:
id(b.auction_params.player_types[0][0]['bidder'])

140286824802624

In [35]:
id(a.auction_params), id(b.auction_params)

(140289819056688, 140286824851296)

In [36]:
x = a.auction_params
id(x)

140289819056688

In [42]:
id(a.get_game()) == id(b.get_game())

True